# Demo 2: Integrate and Analyze
This demo run a user-specified analysis that takes GDDP data descriptors and user-provided data as arguments. DataSpaces downloads any necessary data and runs the user-provided function. Finally, the result is returned to the user.

In [1]:
from dx_interface import DXInterface
from wet_bulb import pressurefromelev
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import numpy as np

Initialize the data interface to a well-known socket.

In [2]:
client = DXInterface('20.84.58.28:8000')

Compute time-independent air pressure values as a function of elevation.

In [3]:
f=nc.Dataset('elev_721x1440.nc'); #geopotential
z=f['z'][:,:,:]/9.81;z_orig=z[0,:,:]; #converted to meters, centered on 180W
lat_cur=np.linspace(-90,90,721); #because elev data file is 721x1440
lon_cur=np.linspace(-180,180,1440);
lat_des=np.linspace(-90,90,600)
lon_des=np.linspace(-180,180,1440);
interp = RegularGridInterpolator((lat_cur, lon_cur), z_orig,bounds_error=False,fill_value=None);
X,Y = np.meshgrid(lat_des,lon_des,indexing='ij')
z_interp=interp((X,Y))
psfc_chunk=pressurefromelev(z_interp[:,:]);
pressure = psfc_chunk

Write the user-computed pressure data to DataSpaces, with a projection and resolution matching GDDP data. These data will be available for download or any future computations.

In [4]:
client.write(variable = 'pressure',
	    model = 'mymodel',
	    geo_resolution = (0.25,0.25),
        projection = 'wgs84',
        geo_offset = (-60.0, -180.0),
	    data = pressure)

Define a function to be run on data in the DataSpaces staging server. `HeatIndex` takes three N-D arrays as arguments, and runs a vectorized kernel element-wise on these arrays. This highlights the pattern of running an element-wise kernel, but any operation performed upon a tuple of N-D arrays is supported. There is no required return value format.

In [6]:
def HeatIndex(t, p, h):
        def foo(tf, v):
            return tf * v
        def HeatIndexKernel(t1, p1, h1):
            return(foo(t1*p1, h1))
        import numpy
        HeatIndexV = numpy.vectorize(HeatIndexKernel)
        return(HeatIndexV(t, p, h))

Specify an argument list to be used for instantiating `HeatIndex`. The first and last argument specify data to be downloaded from Planetary Comptuer (or loaded from the DataSpaces cache, if available). The middle argument is the previously uploaded pressure data. 

In [5]:
argT = DXInterface.build_arg(source = 'planetary-gddp',
		variable = 'tas',
		model = 'ACCESS-ESM1-5',
        start_date = '1982-11-28',
        end_date = '1982-11-29',
        geo_lb = (38.9,-77.0),
        geo_ub = (40.7,-74.0))
argP = DXInterface.build_arg(source = 'local',
        variable = 'pressure',
        model = 'mymodel',
        geo_lb = (38.9,-77.0),
        geo_ub = (40.7,-74.0))
argH = DXInterface.build_arg(source = 'planetary-gddp',
        variable = 'huss',
        model = 'ACCESS-ESM1-5',
        start_date = '1982-11-28',
        end_date = '1982-11-29',
        geo_lb = (38.9,-77.0),
        geo_ub = (40.7,-74.0))

In [ ]:
Remotely execute `HeatIndex` in DataSpaces. The Dat

In [7]:
result = client.exec(HeatIndex, [argT, argP, argH])
print(result)

[[[38889.28127712 39212.85871056 38807.02065091 38504.72922208
   37683.30568699 37609.37981798 38406.24723223 38878.06069059
   38870.64759835 40246.09822265 40681.9028031  31034.08453625
   37268.57844765]
  [36483.24317631 37968.86408037 38164.7865609  37267.78717075
   37218.24057993 37101.2146475  37689.37289099 38125.61835493
   38468.37294588 39186.51638512 38455.40987325 36785.64680145
   35523.12045746]
  [38301.27022088 37687.41002838 37362.61028688 37207.02139914
   36928.433962   36747.38163183 37899.0223677  38908.83295487
   38400.38548756 38979.78546854 38357.3168261  38563.09895008
   38993.05618419]
  [36863.32890374 37958.49321024 36450.39435811 36154.89018729
   36238.81905727 36208.58558771 37386.87192959 37427.67045563
   36399.63563548 37205.54351108 37824.28990322 37410.41407359
   38043.93528295]
  [31717.44559464 31788.09525367 32834.81571647 34807.22411432
   36235.34291266 34584.2854698  33689.50270051 33381.4413144
   33634.67565547 35477.16850448 36852.6692